<a href="https://colab.research.google.com/github/tumblingdice512/Research/blob/master/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import jax
import numpy as np
import random

In [55]:
#モデルについて
#幅N、深さL+1(l=0,1,･･･,L)のネットワーク
#各パーセプトロンはM成分を持つスピンで各成分は+1か-1を取る
#第l層と第l+1層の各パーセプトロンの同一成分は全て結合→重みJ
#各層の重みJはΣJ**2 = Nと正規化されている
#第l層のスピンの値に重みをかけた値に対して符号関数をかませたものが第l+1層の値(0に対しては+1か-1にする)
#初期条件としては、入力層と出力層のスピンの値を与える
#
#
#
#
#

In [56]:
N = 20
L = 5

In [57]:
def sgn(x):
  if x != 0:
    return np.sign(x)

  else:
    return 1


In [58]:
print(sgn(-5))
print(sgn(0))
print(sgn(3))
#符号関数が定義できているか確認

-1
1
1


In [59]:
a = [1, -1]
b = random.choice(a)

print(b)

-1


In [60]:
S0 = np.array([random.choice([1,-1]) for i in range(N)])
SL = np.array([random.choice([1,-1]) for i in range(N)])

print(S0)
print("S0の要素数は",len(S0))

print(SL)
print("SLの要素数は",len(SL))

#初期条件として与えるスピンを生成
#各パーセプトロンのスピンの成分数を増やすときは、SOとSLを行列の形式にすればよい。

[ 1  1 -1  1 -1 -1  1  1  1  1  1 -1 -1 -1  1  1 -1 -1  1 -1]
S0の要素数は 20
[ 1 -1  1 -1  1  1  1  1 -1  1 -1  1 -1  1  1  1 -1  1 -1  1]
SLの要素数は 20


In [66]:
#S0 = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])
#SL = np.array([[random.choice([1,-1]) for i in range(N)],[random.choice([1,-1]) for i in range(N)]])

#print(S0)

#print(SL)

#S0とSLの成分数が2の時

[[-1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1  1 -1  1]
 [ 1  1  1  1  1 -1 -1  1  1  1  1 -1  1  1 -1  1 -1  1  1 -1]]
[[ 1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1]
 [ 1 -1 -1 -1 -1  1  1 -1  1  1  1  1  1  1 -1 -1 -1 -1 -1  1]]
